# 第8章 数据规整：聚合、合并和重塑

## 层次化索引

In [2]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt

层次化索引（hierarchical indexing）是pandas的⼀项重要功能，它使你能在⼀个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理⾼维度数据。我们先来看⼀个简单的例⼦：创建⼀个Series，并⽤⼀个由列表或数组组成的列表作为索引

In [4]:
data=pd.Series(np.random.randn(6),index=[['a','a','a','b','b','c'],[1,2,3,1,3,1]])
data

a  1   -0.346185
   2   -2.206034
   3   -0.279126
b  1    0.155409
   3   -0.506377
c  1    0.470384
dtype: float64

对于⼀个层次化索引的对象，可以使⽤所谓的部分索引，使⽤它
选取数据⼦集的操作更简单

In [5]:
data['b']

1    0.155409
3   -0.506377
dtype: float64

In [6]:
data.loc[['a','b']]

a  1   -0.346185
   2   -2.206034
   3   -0.279126
b  1    0.155409
   3   -0.506377
dtype: float64

In [8]:
data['a':'b']

a  1   -0.346185
   2   -2.206034
   3   -0.279126
b  1    0.155409
   3   -0.506377
dtype: float64

甚⾄还可以在“内层”中进⾏选取

In [11]:
data.loc[:,3]

a   -0.279126
b   -0.506377
dtype: float64

对于⼀个DataFrame，每条轴都可以有分层索引：

In [17]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),
                   index=[['a','a','b','b'],[1,2,1,2]],
                   columns=[['Ohio','Ohio','Colorado'],['Green','red','green']])
frame

Ohio     Colorado
    Green red    green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [21]:
frame.index.names=['key1','key2']
frame.columns.names=['state','color']
frame

state      Ohio     Colorado
color     Green red    green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
frame['Ohio']

color      Green  red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [ ]:
MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green'names=['state','color'])

## 重排与分级排序

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别
上的值对数据进⾏排序。 swaplevel接受两个级别编号或名称，
并返回⼀个互换了级别的新对象（但数据不会发⽣变化）

In [24]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green red    green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

⽽sort_index则根据单个级别中的值对数据进⾏排序。交换级别
时，常常也会⽤到sort_index，这样最终结果就是按照指定顺序
进⾏字⺟排序了

In [30]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green red    green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [32]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green red    green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 根据级别汇总统计

许多对DataFrame和Series的描述和汇总统计都有⼀个level选
项，它⽤于指定在某条轴上求和的级别。再以上⾯那个
DataFrame为例，我们可以根据⾏或列上的级别来进⾏求和

In [33]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green red    green
key2                    
1         6   8       10
2        12  14       16

In [34]:
frame.sum(level='color',axis=1)

color      Green  red  green
key1 key2                   
a    1         0    1      2
     2         3    4      5
b    1         6    7      8
     2         9   10     11

这其实是利⽤了pandas的groupby功能，本书稍后将对其进⾏详
细讲解

## 使⽤DataFrame的列进⾏索引

⼈们经常想要将DataFrame的⼀个或多个列当做⾏索引来⽤，或
者可能希望将⾏索引变成DataFrame的列。以下⾯这个
DataFrame为例

In [41]:
frame2=pd.DataFrame({'a':range(7),'b':range(7),
'c':['one','one','one','two','two','two','two'],
'd':[0,1,2,0,1,2,3]})
frame2

,a,b,c,d
0,0,0,one,0
1,1,1,one,1
2,2,2,one,2
3,3,3,two,0
4,4,4,two,1
5,5,5,two,2
6,6,6,two,3


DataFrame的set_index函数会将其⼀个或多个列转换为⾏索
引，并创建⼀个新的DataFrame：

In [43]:
frame3=frame2.set_index(['c','d'])
frame3

a  b
c   d      
one 0  0  0
    1  1  1
    2  2  2
two 0  3  3
    1  4  4
    2  5  5
    3  6  6

默认情况下，那些列会从DataFrame中移除，但也可以将其保留
下来

In [44]:
frame2.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  0  one  0
    1  1  1  one  1
    2  2  2  one  2
two 0  3  3  two  0
    1  4  4  two  1
    2  5  5  two  2
    3  6  6  two  3

reset_index的功能跟set_index刚好相反，层次化索引的级别会
被转移到列⾥⾯

In [45]:
frame3.reset_index()

,c,d,a,b
0,one,0,0,0
1,one,1,1,1
2,one,2,2,2
3,two,0,3,3
4,two,1,4,4
5,two,2,5,5
6,two,3,6,6


## 合并数据集

pandas对象中的数据可以通过⼀些⽅式进⾏合并：
pandas.merge可根据⼀个或多个键将不同DataFrame中的⾏
连接起来。 SQL或其他关系型数据库的⽤户对此应该会⽐较
熟悉，因为它实现的就是数据库的join操作。
pandas.concat可以沿着⼀条轴将多个对象堆叠到⼀起。
实例⽅法combine_first可以将重复数据编接在⼀起，⽤⼀个对
象中的值填充另⼀个对象中的缺失值

### 数据库⻛格的DataFrame合并

数据集的合并（merge）或连接（join）运算是通过⼀个或多个
键将⾏链接起来的。这些运算是关系型数据库（基于SQL）的核
⼼。 pandas的merge函数是对数据应⽤这些算法的主要切⼊点

In [50]:
df1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2=pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [51]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


这是⼀种多对⼀的合并。 df1中的数据有多个被标记为a和b的
⾏，⽽df2中key列的每个值则仅对应⼀⾏。对这些对象调⽤
merge即可得到：

In [52]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


注意，我并没有指明要⽤哪个列进⾏连接。如果没有指定，
merge就会将重叠列的列名当做键。不过，最好明确指定⼀下：

In [53]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果两个对象的列名不同，也可以分别进⾏指定：

In [54]:
df3=pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4=pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [55]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [56]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


可能你已经注意到了，结果⾥⾯c和d以及与之相关的数据消失
了。默认情况下， merge做的是“内连接”；结果中的键是交集。
其他⽅式还有"left"、 "right"以及"outer"。外连接求取的是键的并
集，组合了左连接和右连接的效果

In [57]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


## 轴向连接

另⼀种数据合并运算也被称作连接（concatenation）、绑定
（binding）或堆叠（stacking）。 NumPy的concatenation函数
可以⽤NumPy数组来做

In [65]:
left1=pd.DataFrame({'key':['a','b','a','a','b'],'value':range(5)})
arr=np.arange(12).reshape((3,4))
arr


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [66]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

## 数据规整：聚合、合并和重塑

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

### 轴向连接

pandas的concat函数提供了一种能够解决这些问题的可靠方式。我将给出一些例子来讲解其使用方式。假设有三个没有重叠索引的Series：

In [3]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

对这些对象调用concat可以将值和索引粘合在一起：

In [4]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）：

In [5]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。传入join='inner'即可得到它们的交集：

In [10]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [11]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [12]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


在这个例子中，f和g标签消失了，是因为使用的是join='inner'选项。

不过有个问题，参与连接的片段在结果中区分不开。假设想要在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的（这里必须要加axis=0）

In [20]:
result = pd.concat([s1, s1, s3], axis=0, keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [21]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头：

In [22]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


同样的逻辑也适用于DataFrame对象：

In [27]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [28]:
df2

,three,four
a,5,6
c,7,8


In [29]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值：

In [24]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

我们可以用names参数命名创建的轴级别：

In [25]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

最后一个关于DataFrame的问题是，DataFrame的行索引不包含任何相关数据：

In [31]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221


In [32]:
df2

,b,d,a
0,0.274992,0.228913,1.352917
1,0.886429,-2.001637,-0.371843


在这种情况下，传入ignore_index=True即可：

In [33]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


### 合并重叠数据

还有一种数据组合问题不能用简单的合并（merge）或连接（concatenation）运算来处理。比如说，你可能有索引全部或部分重叠的两个数据集。
举个有启发性的例子，我们使用NumPy的where函数，它表示一种等价于面向数组的if-else：

In [35]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [36]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

满足条件输出b不满足条件输出a

In [38]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series有一个combine_first方法，实现的也是一样的功能，还带有pandas的数据对齐：

In [39]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

组合两个 DataFrames，如果第一个 DataFrames 具有空值，则使用第二个 DataFrames 中的数据
用传递对象中的数据为调用对象的缺失数据“打补丁”

In [40]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


##  重塑和轴向旋转

有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。

###  重塑层次化索引
层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。主要功能有二：

- stack：将数据的列“旋转”为行。
- unstack：将数据的行“旋转”为列。

我将通过一系列的范例来讲解这些操作。接下来看一个简单的DataFrame，其中的行列索引均为字符串数组：

In [44]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


对该数据使用stack方法即可将列转换为行，得到一个Series：

In [45]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

对于一个层次化索引的Series，你可以用unstack将其重排为一个DataFrame：

In [46]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，unstack操作的是最内层（stack也是如此）。传入分层级别的编号或名称即可对其它级别进行unstack操作：

In [47]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [48]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据：

In [50]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack默认会滤除缺失数据，因此该运算是可逆的：

In [53]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [54]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别：

In [56]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [57]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

当调用stack，我们可以指明轴的名字：

In [58]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

###  将“长格式”旋转为“宽格式”
多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。先加载一些示例数据，做一些时间序列规整和数据清洗：

In [71]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [72]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


这就是多个时间序列（或者其它带有两个或多个键的可观察数据，这里，我们的键是date和item）的长格式。表中的每行代表一次观察。

关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有一个好处：随着表中数据的添加，item列中的值的种类能够增加。在前面的例子中，date和item通常就是主键（用关系型数据库的说法），不仅提供了关系完整性，而且提供了更为简单的查询支持。有的情况下，使用这样的数据会很麻烦，你可能会更喜欢DataFrame，不同的item值分别形成一列，date列中的时间戳则用作索引。DataFrame的pivot方法完全可以实现这个转换：

In [73]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充DataFrame的数据列。假设有两个需要同时重塑的数据列：

In [79]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.393811
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.132037
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.529250
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.303550
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.889604
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.173346
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.643302
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.254008
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.117248
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.866544


如果忽略最后一个参数，得到的DataFrame就会带有层次化的列：

In [80]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.132037  0.393811   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.889604  0.303550   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.254008  0.643302   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.079464  0.866544   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.558011  1.487506   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.529250  
1959-06-30 23:59:59.999999999 -0.173346  
1959-09-30 23:59:59.999999999  0.117248  
1959-12-31 23:59:59.999999999 -0.397475  
1960-03-31 23:59:59.999999999  0.614350

In [81]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


pivot其实就是用set_index创建层次化索引，再用unstack重塑：

In [82]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.132037  0.393811   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.889604  0.303550   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.254008  0.643302   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.079464  0.866544   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.558011  1.487506   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.271959  0.427964   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.315273 -0.508070   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.529250  
1959-06-30 23:59:59.999999999 -0.173346  
1959-09-30 23:59:59.999999999  0.117248  
1959-12-31 23:59:59.999999999 -0.397475  
1960-03-31 23:59:59.999999999  0.614350  
1960-06-30 23:59:59.999999999  0.738759  
1960-09-30 23:59:59.999999999  0.445930

###  将“长格式”旋转为“宽格式”
多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。先加载一些示例数据，做一些时间序列规整和数据清洗：

In [71]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [72]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


这就是多个时间序列（或者其它带有两个或多个键的可观察数据，这里，我们的键是date和item）的长格式。表中的每行代表一次观察。

关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有一个好处：随着表中数据的添加，item列中的值的种类能够增加。在前面的例子中，date和item通常就是主键（用关系型数据库的说法），不仅提供了关系完整性，而且提供了更为简单的查询支持。有的情况下，使用这样的数据会很麻烦，你可能会更喜欢DataFrame，不同的item值分别形成一列，date列中的时间戳则用作索引。DataFrame的pivot方法完全可以实现这个转换：

In [73]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充DataFrame的数据列。假设有两个需要同时重塑的数据列：

In [79]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.393811
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.132037
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.529250
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.303550
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.889604
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.173346
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.643302
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.254008
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.117248
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.866544


如果忽略最后一个参数，得到的DataFrame就会带有层次化的列：

In [80]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.132037  0.393811   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.889604  0.303550   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.254008  0.643302   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.079464  0.866544   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.558011  1.487506   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.529250  
1959-06-30 23:59:59.999999999 -0.173346  
1959-09-30 23:59:59.999999999  0.117248  
1959-12-31 23:59:59.999999999 -0.397475  
1960-03-31 23:59:59.999999999  0.614350

In [81]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


pivot其实就是用set_index创建层次化索引，再用unstack重塑：

In [82]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.132037  0.393811   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.889604  0.303550   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.254008  0.643302   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.079464  0.866544   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.558011  1.487506   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.271959  0.427964   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.315273 -0.508070   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.529250  
1959-06-30 23:59:59.999999999 -0.173346  
1959-09-30 23:59:59.999999999  0.117248  
1959-12-31 23:59:59.999999999 -0.397475  
1960-03-31 23:59:59.999999999  0.614350  
1960-06-30 23:59:59.999999999  0.738759  
1960-09-30 23:59:59.999999999  0.445930

### 将“宽格式”旋转为“长格式”
旋转DataFrame的逆运算是pandas.melt。它不是将一列转换到多个新的DataFrame，而是合并多个列成为一个，产生一个比输入长的DataFrame。看一个例子：

In [83]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


key列可能是分组指标，其它的列是数据值。当使用pandas.melt，我们必须指明哪些列是分组指标。下面使用key作为唯一的分组指标：

In [84]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使用pivot，可以重塑回原来的样子：

In [85]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


因为pivot的结果从列创建了一个索引，用作行标签，我们可以使用reset_index将数据移回列：

In [91]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


还可以指定列的子集，作为值的列

In [92]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandas.melt也可以不用分组指标：

In [93]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
